In [ ]:
from PIL import Image, ImageDraw
from IPython.display import display
from tqdm.notebook import tqdm

In [ ]:
insert_size = 128
border = 2

scenes = ['car', 'chess', 'dragon', 'sponza']

methods = [['reference', '48x560-interp', '48x560-naive', '48x560-oX', '48x560-o1', '48x560-o2', '48x560-o3'],
           ['reference', '36x420-interp', '36x420-naive', '36x420-oX', '36x420-o1', '36x420-o2', '36x420-o3'],
           ['reference', '24x280-interp', '24x280-naive', '24x280-oX', '24x280-o1', '24x280-o2', '24x280-o3'],
           ['reference', '12x140-interp', '12x140-naive', '12x140-oX', '12x140-o1', '12x140-o2', '12x140-o3']]

coords = [[(355, 615), (  5, 455), (770, 390)],
          [(473, 455), (660, 315), (300, 370)],
          [(350, 360), (535, 540), (540, 385)],
          [(270, 250), (440, 598), (490, 450)]]

colors = ['#2ca02c', '#1f77b4', '#ff7f0e']

n_scenes = len(scenes)
n_res = len(methods)
n_methods = len(methods[0])
n_inserts = len(colors)
n_f = 3

images = []
inserts = []

with tqdm(total=n_scenes * n_f * n_res * n_methods * n_inserts) as pbar:
    for scene_index in range(n_scenes):
        images.append([])
        inserts.append([])
        for f_index in range(n_f):
            images[scene_index].append([])
            inserts[scene_index].append([])
            for res_index in range(n_res):
                images[scene_index][f_index].append([])
                inserts[scene_index][f_index].append([])
                for method_index in range(n_methods):
                    with Image.open('figures/image-{}-f{}-{}.png'.format(scenes[scene_index], f_index+1, methods[res_index][method_index])) as img:
                        images[scene_index][f_index][res_index].append(img)
                        inserts[scene_index][f_index][res_index].append([])

                        for insert_index in range(n_inserts):
                            x, y = coords[scene_index][insert_index]
                            insert = img.crop((x, y, x + insert_size, y + insert_size))
                            inserts[scene_index][f_index][res_index][method_index].append(insert)

                            ImageDraw.Draw(img).rectangle((x, y, x + insert_size - 1, y + insert_size - 1), outline=colors[insert_index], width=border)
                            ImageDraw.Draw(insert).rectangle((0, 0, insert_size - 1, insert_size - 1), outline=colors[insert_index], width=border)
                            pbar.update()

                        img.save('figures/insert-{}-f{}-{}.png'.format(scenes[scene_index], f_index+1, methods[res_index][method_index]))

In [ ]:
print('References with inserts')
print('Columns: Near focus, Mid focus, Far focus')

img = Image.new(mode='RGB', size=(3 * 1024, 1024))

for scene_index in range(n_scenes):
    for f_index in range(n_f):
        img.paste(images[scene_index][f_index][0][0], (f_index * 1024, 0))
    display(img)

In [ ]:
img = Image.new(mode='RGB', size=(n_f * (insert_size-border) + border, n_methods * (insert_size-border) + border))

for scene_index in range(n_scenes):
    for insert_index in range(n_inserts):
        for res_index in range(n_res):
            for f_index in range(n_f):
                for method_index in range(n_methods):
                    img.paste(inserts[scene_index][f_index][res_index][method_index][insert_index], (f_index * (insert_size-border), method_index * (insert_size-border)))
            print('\n[Methods x Focus]    Scene: {}    Insert: {}    Resolution: {}'.format(scenes[scene_index], insert_index+1, res_index+1))
            img.save('figures/insert-{}-{}-i{}.png'.format(scenes[scene_index], methods[res_index][method_index], insert_index+1))
            display(img)

In [ ]:
img = Image.new(mode='RGB', size=(n_res * (insert_size-border) + border, n_methods * (insert_size-border) + border))

for scene_index in range(n_scenes):
    for insert_index in range(n_inserts):
        for f_index in range(n_f):
            for res_index in range(n_res):
                for method_index in range(n_methods):
                    img.paste(inserts[scene_index][f_index][res_index][method_index][insert_index], (res_index * (insert_size-border), method_index * (insert_size-border)))
            print('\n[Methods x Resolutions]    Scene: {}    Insert: {}    Focus: {}'.format(scenes[scene_index], insert_index+1, f_index+1))
            img.save('figures/insert-{}-f{}-i{}.png'.format(scenes[scene_index], f_index+1, insert_index+1))
            display(img)

In [ ]:
spectrum_size = 2 * insert_size

img = Image.new(mode='RGB', size=(n_methods * spectrum_size, n_res * spectrum_size))

for scene_index in range(n_scenes):
    for f_index in range(n_f):
        for res_index in range(n_res):
            for method_index in range(n_methods):
                with Image.open('figures/spectrum-{}-f{}-{}.png'.format(scenes[scene_index], f_index+1, methods[res_index][method_index])) as spectrum:
                    img.paste(spectrum.resize((spectrum_size, spectrum_size)), (method_index * spectrum_size, res_index * spectrum_size))
        print('\n[Resolutions x Methods]    Scene: {}    Focus: {}'.format(scenes[scene_index], f_index+1))
        img.save('figures/spectrum-{}-f{}.png'.format(scenes[scene_index], f_index+1))
        display(img)